In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import urllib
import random
import cv2
import os
from collections import OrderedDict

In [2]:
# just run this cell
FLOWERS_DIR = './flower_photos'
TRAIN_FRACTION = 0.8
RANDOM_SEED = 2018


def download_images():
  """If the images aren't already downloaded, save them to FLOWERS_DIR."""
  if not os.path.exists(FLOWERS_DIR):
    DOWNLOAD_URL = 'http://download.tensorflow.org/example_images/flower_photos.tgz'
    print('Downloading flower images from %s...' % DOWNLOAD_URL)
    urllib.request.urlretrieve(DOWNLOAD_URL, 'flower_photos.tgz')
    !tar xfz flower_photos.tgz
  print('Flower photos are located in %s' % FLOWERS_DIR)


def make_train_and_test_sets():
    train_examples, val_examples, test_examples = [], [], []
    shuffler = random.Random(111)
    is_root = True
    for (dirname, subdirs, filenames) in os.walk(FLOWERS_DIR):
        #print('dirname:  ', dirname)
        #print('subdirs:   ',subdirs)
        #print('filenames:  ',filenames)
        if is_root:
            subdirs = sorted(subdirs)
            #print(subdirs)
            classes = OrderedDict(enumerate(subdirs))
            #print(classes)
            label_to_class = dict([(x,i) for i, x in enumerate(subdirs)])
            #print(label_to_class)
            is_root = False
        else:
            filenames.sort()
            #print('filenames2:  ',filenames)
            shuffler.shuffle(filenames)
            full_filenames = [os.path.join(dirname, f) for f in filenames]
            #print(full_filenames)
            label = dirname.split('/')[-1] # \\
            print("LABEL:  ",label)
            label_class = label_to_class[label]
            #print(label_class)

            examples = list(zip(full_filenames, [label_class] * len(filenames)))
            #print(examples)
            num_train = int(len(filenames) * 0.8)
            num_valid = int((len(filenames)- num_train)/2)
            #print(num_train)
            train_examples.extend(examples[:num_train])
            test_examples.extend(examples[num_train:num_train + num_valid])
            val_examples.extend(examples[num_train + num_valid:])

    shuffler.shuffle(train_examples)
    shuffler.shuffle(test_examples)
    return train_examples, val_examples, test_examples, classes

In [3]:
download_images()
TRAIN_EXAMPLES, VAL_EXAMPLES, TEST_EXAMPLES, CLASSES = make_train_and_test_sets()
NUM_CLASSES = len(CLASSES)

print('\nThe dataset has %d label classes: %s' % (NUM_CLASSES, CLASSES.values()))
print('There are %d training images' % len(TRAIN_EXAMPLES))
print('there are %d validation image' % len(VAL_EXAMPLES))
print('there are %d test images' % len(TEST_EXAMPLES))

Flower photos are located in ./flower_photos
LABEL:   roses
LABEL:   tulips
LABEL:   sunflowers
LABEL:   dandelion
LABEL:   daisy

The dataset has 5 label classes: odict_values(['daisy', 'dandelion', 'roses', 'sunflowers', 'tulips'])
There are 2934 training images
there are 369 validation image
there are 367 test images


In [4]:
train_ds  = TRAIN_EXAMPLES
val_ds = VAL_EXAMPLES
test_ds = TEST_EXAMPLES

def get_generator(ds):
    def data_generator():
        for i in range(len(ds)):
            path, label = ds[i]
            img = cv2.imread(path)
            img_resized = cv2.resize(img, (224,224))/255
            yield img_resized, (label,)
    return data_generator

train_ds = tf.data.Dataset.from_generator(get_generator(train_ds), output_types=(tf.float32, tf.uint8), output_shapes=((224,224,3),(1,)))
train_ds = train_ds.shuffle(100)
train_ds = train_ds.batch(64)

val_ds = tf.data.Dataset.from_generator(get_generator(val_ds), output_types=(tf.float32, tf.uint8), output_shapes=((224,224,3),(1,)))
val_ds = val_ds.batch(64)

test_ds = tf.data.Dataset.from_generator(get_generator(test_ds), output_types=(tf.float32, tf.uint8), output_shapes=((224,224,3),(1,)))
test_ds = test_ds.batch(64)

In [5]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape = [224, 224,3]),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(64, (2, 2), activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(64, (2, 2), activation='relu'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(5, activation ='softmax')
])

In [6]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [7]:
es = tf.keras.callbacks.EarlyStopping(monitor="val_accuracy", patience=3, restore_best_weights=True)
model.fit(train_ds, validation_data=val_ds, epochs = 15, callbacks=[es])

Epoch 1/15
46/46 [==============================] - 280s 6s/step - loss: 1.8172 - accuracy: 0.3112 - val_loss: 1.2551 - val_accuracy: 0.4743
Epoch 2/15
46/46 [==============================] - 265s 6s/step - loss: 1.1097 - accuracy: 0.5545 - val_loss: 1.0558 - val_accuracy: 0.5583
Epoch 3/15
46/46 [==============================] - 255s 6s/step - loss: 0.8709 - accuracy: 0.6595 - val_loss: 1.1129 - val_accuracy: 0.5583
Epoch 4/15
46/46 [==============================] - 265s 6s/step - loss: 0.5970 - accuracy: 0.7883 - val_loss: 1.3300 - val_accuracy: 0.5285
Epoch 5/15
46/46 [==============================] - 257s 6s/step - loss: 0.3492 - accuracy: 0.8913 - val_loss: 1.7073 - val_accuracy: 0.5474


In [8]:
evaluation_results = model.evaluate(test_ds)

print(f'Your accuracy: {evaluation_results[1]*100}%')

6/6 [==============================] - 9s 1s/step - loss: 1.0672 - accuracy: 0.5858
Your accuracy: 58.583104610443115%


Add BatchNormalization and Dropout

In [9]:
from keras.layers.normalization.batch_normalization import BatchNormalization
# Batch norm

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape = [224, 224,3]),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.BatchNormalization(),
    # norm
    tf.keras.layers.Conv2D(64, (2, 2), activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.BatchNormalization(),
    # norm
    tf.keras.layers.Conv2D(64, (2, 2), activation='relu'),
    tf.keras.layers.BatchNormalization(),
    # norm
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dropout(0.4),
    # dropout
    tf.keras.layers.Dense(5, activation ='softmax'),
    tf.keras.layers.Dropout(0.4)

    # dropout
])

In [10]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
es = tf.keras.callbacks.EarlyStopping(monitor="val_accuracy", patience=3, restore_best_weights=True)
model.fit(train_ds, validation_data=val_ds, epochs = 15, callbacks=[es])

Epoch 1/15
46/46 [==============================] - 309s 7s/step - loss: 6.8078 - accuracy: 0.2928 - val_loss: 10.6794 - val_accuracy: 0.2764
Epoch 2/15
46/46 [==============================] - 295s 6s/step - loss: 6.9430 - accuracy: 0.2808 - val_loss: 11.9414 - val_accuracy: 0.2466
Epoch 3/15
46/46 [==============================] - 295s 6s/step - loss: 6.8891 - accuracy: 0.2900 - val_loss: 11.8620 - val_accuracy: 0.2547
Epoch 4/15
29/46 [=================>............] - ETA: 1:41 - loss: 7.1389 - accuracy: 0.2850

In [ ]:
evaluation_results = model.evaluate(test_ds)

print(f'Your accuracy: {evaluation_results[1]*100}%')